О содержании статьи.

Об интерпретации моделей машинного обучения в целом (кратко).

Локальное упрощение и оценка важности признаков в упрощенной модели (иллюстрация).

Введение в LIME и SHAP

Схема из кругов: интерпретация моделей со сложными признаками, с простыми признаками (переход - выбор конкретного примера, т. е. локальная интерпретация), LIME, SHAP, библиотека SHAP, saliency maps, mean impurity decrease. LIME - подход к упрощению модели, SHAP - подход к анализу вклада признаков в модели, в котором для вычислительной эффективности требуется простота модели. Интерпретация сложной модели = локальное упрощение модели + оценка важности признаков в простой модели. LIME - подход к упрощению модели, а SHAP отвечает на вопрос о том, как лучше всего оценивать важность признаков в простой модели. Получение модели с простыми признаками.

Локальное упрощение модели: терминология, обозначения и примеры

Алгоритм LIME

Shapley additive explanation: вклад признаков в модель
(+ интересное замечание: анализ не только моделей, но и самой целевой зависимости)

Linear LIME + kernel SHAP

SHAP на практике

https://arxiv.org/pdf/2002.11097.pdf

https://papers.nips.cc/paper/2020/file/0d770c496aa3da6d2c3f2bd19e7b9d6b-Paper.pdf

https://papers.nips.cc/paper/2020/file/32e54441e6382a7fbacbbbaf3c450059-Paper.pdf

-----

(Введение, о содержании статьи, ссылки на работы)

## Интерпретация моделей машинного обучения

Модели машинного обучения (такие как нейронные сети, машины опорных векторов, ансамбли решающих деревьев) являются "прозрачными" в том смысле, что все происходящие внутри них вычисления известны. Но тем не менее часто говорят, что модели машинного обучения плохо интерпретируемы. Здесь имеется в виду то, что процесс принятия решения не удается представить в понятной человеку форме, то есть:

1. Понять, какие признаки или свойства входных данных влияют на ответ
2. Разложить алгоритм принятия решения на понятные составные части
3. Объяснить смысл промежуточных результатов, если они есть
3. Описать в текстовом виде алгоритм принятия решения (возможно, с привлечением схем или графиков)

Достичь полной интерпретируемости в машинном обучении, как правило, не удается, но даже частичная интерпретация может существенно помочь. Если мы узнаем, на что именно обращает внимание алгоритм, какими правилами руководствуется, то сможем оценить правдоподобность этих правил и тем самым распознать переобучение. Также мы сможем лучше понять, почему модель ошибается на тех или иных примерах, что может помочь улучшить алгоритм обучения. Кроме того, интерпретировав алгоритм, мы можем открыть для себя что-то новое о свойствах исследуемых данных (например, какие признаки в табличных данных в наибольшей степени влияют на ответ).

> Humans usually have prior knowledge about the application domain, which they can use to accept (trust) or reject a prediction if they understand the reasoning behind it. ([Ribeiro et al., 2016]($Why Should I Trust You?: Explaining the Predictions of Any Classifier$))

Обзор способов интерпретации моделей машинного обучения можно найти, например, в [Linardatos et al., 2020]($Explainable AI: A Review of Machine Learning Interpretability Methods$) и [Li et al., 2021]($Interpretable Deep Learning: Interpretation, Interpretability, Trustworthiness, and Beyond$). 

### Интерпретация моделей для оценки качества

Авторы методa LIME ([Ribeiro et al., 2016]($Why Should I Trust You?: Explaining the Predictions of Any Classifier$)) делают акцент на ситуации, когда требуется оценить качество обученной модели (например, чтобы сделать выбор между одной из нескольких моделей). Оценка точности на тестовой выборке не всегда позволяет хорошо оценить качество из-за следующих проблем:

1. <p><b>Сдвиг данных</b> (data shift). Эта ситуация означает, что данные, на которых модель будет применяться, среднестатистически отличаются от тех, на которых модель обучалась и тестировалась, то есть распределение входных данных отличается при тестировании и применении: $P_{test}(X, Y) \neq P_{usage}(X, Y)$. Поскольку $P(X, Y) = P(Y|X)P(X)$, то двумя вариантами сдвига данных является сдвиг в исходных данных $P(X)$ и сдвиг целевой переменной $P(Y|X)$.</p>
<p>Одна из проблем сдвига данных в том, что большинство метрик качества (accuracy, MSE, logloss, F1 и другие) зависят от распределения исходных данных $P(X)$, то есть, упрощенно говоря, от соотношения разных типов примеров в датасете. Например, пусть модель тестировалась на датасете, в котором 80% изображений были высокого качества (HQ), а применяться будет в условиях, когда, наоборот, 80% изображений будут низкого качества (LQ). Пусть мы сравниваем две модели: на HQ-изображениях точность первой модели лучше, чем второй, а на LQ-изображениях, наоборот, точность второй модели лучше, чем первой. Если при тестировании большая часть изображений были HQ, то мы сделаем вывод, что первая модель лучше, тогда как на самом деле лучше была бы вторая.</p>  
<p>Другая проблема сдвига данных в том, что в обучающих и тестовых данных могут присутствовать такие корреляции, которые не обобщаются на другие выборки. Например, мы классифицируем животных по изображению, но большинство изображений рыб, которые у нас имеются, содержат также пальцы рыбака (эти изображения мы используем как при обучении, так и при тестировании). Модель может научиться классифицировать как рыбу изображение, содержащее пальцы, что в целом неверно и может не работать на других выборках (см. также <a href=$Inductive Biases for Deep Learning of Higher-Level Cognition$>Goyal and Bengio, 2020</a> и обзор <a href=$Априорные гипотезы и регуляризация в машинном обучении$>Априорные гипотезы и регуляризация в машинном обучении</a>; пример с рыбами см. в <a href=$Approximating CNNs with Bag-of-local-Features models works surprisingly well on ImageNet$>Brendel and Bethge, 2019</a>).</p>

2. **Утечка данных** (data leakage, или target leakage). Например, ID пациента может сильно коррелировать с диагнозом, но только в текущем датасете (который поделен на обучающую и тестовую часть). Модель, предсказывающая диагноз по ID, будет иметь высокую точность на тестовом датасете, но в целом очевидно, что в данной задаче такой способ предсказания некорректен и не будет хорошо работать на других данных. Утечка данных является частным случаем сдвига данных, поскольку зависимость ID $\to$ диагноз была в $P_{train}(X, Y)$ и $P_{test}(X, Y)$, но ее не будет в $P_{usage}(X, Y)$. Избавиться от утечки данных не всегда просто.

> Еxample for this is KDD-Cup 2008 breast cancer prediction competition, where the patient ID contained an obvious leak. It is by no means obvious that removing this feature would leave a leakage-free dataset, however. Assuming different ID ranges correspond to different health care facilities (in different geographical locations, with different equipment), there may be additional traces of this in the data. If for instance the imaging equipment's grey scale is slightly different and in particular grey levels are higher in the location with high cancer rate, the model without ID could pick up this leaking signal from the remaining data, and the performance estimate
would still be optimistic (the winners show evidence of this in their report). ([Kaufman et al., 2011]($Leakage in Data Mining: Formulation, Detection, and Avoidance$))

Если удастся интерпретировать модель (хотя бы приблизительно), то мы получим дополнительную информацию, которая поможет надежнее оценить ее качество в условиях возможной утечки и сдвига данных.

### Локальная интерпретация моделей

Вместо попыток интерпретировать модель целиком, что может быть очень сложно, мы можем рассмотреть задачу интерпретации ответа модели $f$ на конкретном, фиксированном примере $x_0$. Например, если на данном изображении модель распознала собаку, то почему она распознала собаку? Какие части и свойства изображения повлияли на предсказание модели?

Для ответа на этот вопрос мы можем изменять $x_0$ и смотреть, как изменится при этом ответ модели, то есть мы изучаем зависимость $f(x_0 + \Delta x)$ от $\Delta x$. При этом возможно удастся с хорошей точностью аппроксимировать эту зависимость простой функцией $g(\Delta x)$. Такой подход называется локальной аппроксимацией модели в окрестности точки $x_0$.

Например, рассчитав градиент $\nabla f(x)$ в точке $x_0$ мы узнаем, как изменится ответ при очень малых изменениях $\Delta x$. При этом мы получаем локальную линейную аппроксимацию (в курсе высшей математики такая аппроксимация называется дифференциалом функции $f$). Такой подход используется, например, при расчете так называемых saliency maps в компьютерном зрении ([Simonyan et al., 2013]($Deep Inside Convolutional Networks: Visualising Image Classification Models and Saliency Maps$)) - производных выходных значений сети по отдельным пикселям изображения. Но такая аппроксимация далеко не всегда адекватна:

1. Производная локальна и не говорит о том, как изменится ответ при существенных изменениях $\Delta x$. Например, если один из признаков достиг состояния "насыщения", то есть значение данного признака более чем достаточно, чтобы сделать какой-то вывод о целевой переменной, то производная по нему почти равна нулю. Эффект будет лишь если мы сильно изменим данный признак.
2. В некоторых моделях (решающих деревьях) производная либо равна нулю, либо не существует.
3. Для бинарных признаков производная не всегда информативна, поскольку малое изменение признака ведет в "невозможную" область нецелого значения, в котором модель и не обязана работать корректно.

Есть и другой подход к локальной интерпретации модели: в качестве $\Delta x$ мы можем рассматривать некий набор осмысленных, не бесконечно малых изменений входных данных. Именно такой подход лежит в основе метода интерпретации LIME, который мы рассмотрим далее.

## LIME: Local Interpretable Model-agnostic Explanations

LIME - это подход к интерпретации ответа модели $f(x_0)$ на конкретном тестовом примере $x_0$ с помощью вычисления значений $f(x_0 + \Delta x)$ для некоторого конечного набора значений $\Delta x$. Иллюстрация работы метода LIME приведена на *рис. 1*. Чтобы формально описать метод LIME, нам понадобится ввести ряд обозначений, которые мы будем использовать и в следующих частях обзора.

<img src="assets/lime2.jpg" width="600" align="center">

<center><i>Рис. 1. Иллюстрация работы метода LIME.</i></center>

Мы интерпретируем модель $f$ на примере $x_0$:

- $f: X \to Y$ - исходная модель
- $x_0 \in X$ - выбранный тестовый пример, предсказание на котором $f(x_0)$ интерпретируется

### Локальное упрощенное представление

Для примера $x_0$ вводим $M$ осмысленных, интерпретируемых изменений $\Delta_i$. Например, для изображений таким изменением может быть удаление отдельного суперпикселя, то есть участка изображения с похожим содержимым и четкими границами (*рис. 1b*). Каждое изменение бинарно, то есть оно либо есть, либо нет. Соответственно, мы получаем $2^M$ различных вариантов $\Delta x$. Наличие или отсутствие каждого из изменений можно описать числом 0 или 1: из этих чисел можно собрать бинарный вектор $z$ размерностью $M$, который будем называть *упрощенным представлением* (*рис. 1c*).

- $\{ \Delta_i \}_{i=1}^M$ - изменения примера $x_0$
- $z_i \in \{0, 1\}$ - индикатор изменения $\Delta_i$
- $z \in \{0, 1\}^M$ - вектор упрощенного представления
- $h: \{0, 1\}^M \to X$ - функция, преобразующая вектор упрощенного представления $z$ в $x_0 + \Delta x$

Например, на *рис. 1* функция $h$ работает следующим образом: все "пристствующие" суперпиксели ($z_i = 1$) рисуются без изменений. Все "отсутствующие" суперпиксели ($z_i = 0$) заполняются белым цветом (либо, как вариант, усредненным цветом соседних суперпикселей). При этом $h([1, 1, \dots, 1]) = x_0$, поскольку вектор из единиц означает отсутствие всех изменений. Фактически выбор функции $h$ равносилен выбору изменений $\Delta_i$, то есть семантики упрощенного представления.

Функцию $h$ мы можем выбрать произвольно, но так, чтобы отдельные изменения $\Delta_i$ были интерпретируемы. При этом мы надеемся, что объяснить предсказание модели на $x_0$ можно интерпретировать, изучая, как влияют на ответ эти изменения. Конечно, функция $g$ может быть выбрана неудачно. Например, если модель определяет стиль фотографии как "ретро" при наличии оттенка "сепия", то изменения отдельных суперпикселей не помогут интерпретировать модель. Если $\Delta_i$ были выбраны неудачно, то всегда можно попробовать заново с другими изменениями $\Delta_i$.

Для табличных данных мы можем в качестве $\Delta_i$ рассматривать замену одного из признаков на ноль, среднее или медианное значение по обучающему датасету.

### Объясняющая модель

Теперь мы можем обучить модель $g$ предсказывать значение $f(h(z))$ по вектору упрощенного представления $z$.  При этом модель $g$ должна быть простой и интерпретируемой (поскольку смысл метода LIME в интерпретации). Например, это может быть линейная модель или решающее дерево. Чтобы обучить модель, нужно собрать обучающую выборку. Для этого нам потребуется получить ответ модели $f$ для разных $z$ (то есть для разных $x_0 + \Delta x$, поскольку $z$ определяет $\Delta x$) и таким образом собрать обучающую выборку для модели $g$.

- $g: \{0, 1\}^M \to Y$ - объясняющая модель
- $\big\{ z^{(i)}, f(h(z^{(i)})) \big\}_{i=1}^N$ - обучающая выборка для объясняющей модели

Максимально возможный размер обучающей выборки равен $2^M$, но обычно $M$ велико, и приходится ограничиться перебором лишь некоторых значений $z$. Авторы предлагают уделять основное внимание таким $z$, которые близки к вектору из единиц: это соответствует небольшим изменениям в $x_0$ (чем больше нулей в $z$, тем больше одновременных изменений $x_0$ мы рассматриваем). Введем функцию $\pi(z)$, определяющую меру близости $h(z)$ к $x_0$, и назначим веса $\pi(z^{(i)})$ примерам из обучающей выборки.

- $\pi: \{0, 1\}^M \to \mathbb{R}$ - мера близости $h(z)$ к $x_0$
- $\big\{ w^{(i)} = \pi(z^{(i)}) \big\}_{i=1}^N$ - веса примеров из обучающей выборки

*Примечание.* В kernel SHAP, который мы рассмотрим в следующих разделах, в качестве $\pi$ берется функция, назначающая большие веса как векторам с большим количеством единиц, так и векторам с большим количеством нулей. Так мы акцентируем внимание в том числе на значениях $z$ с большим количеством нулей, то есть на отдельных интерпретируемых компонентах (напрмер, отдельных суперпикселях в случае изображений).

Для обучения модели $g$ осталось выбрать функцию потерь - например, среднеквадратичное отклонение. Эта функция будет сравнивать предсказания моделей $f$ и $g$. Авторы предлагают также использовать "штраф за сложность" $\Omega(g)$ - например, количество ненулевых весов в линейной модели. Если в качестве функции потерь выбрано среднеквадратичное отклонение, то задача оптимизации формулируется следующим образом:

$\sum\limits_{i=1}^N w^{(i)} \Big( g(z^{(i)}) - f(h(z^{(i)})) \Big)^2 + \Omega(g) \to \min\limits_g$

В данной формуле мы считаем квадрат разности предсказаний объясняющей модели $g(z^{(i)})$ и исходной модели $f(h(z^{(i)}))$, и считаем взвешенную сумму по обучающей выборке, используя веса $w^{(i)}$. Кроме того мы прибавляем штраф за сложность объясняющей модели $\Omega(g)$.

### Резюме

Суть подхода LIME в том, что мы аппроксимируем предсказание модели $f$ в окрестности тестового примера $x_0$ более простой, легко интерпретируемой моделью $g$, которая использует упрощенное представление $z$. Например, если модель $g$ линейна, то каждому изменению $\Delta_i$ (например, суперпикселю в изображении) сопоставляется некий вес. 

При этом мы надеемся, что такая аппроксимация адекватна, то есть наличие $i$-го изменения линейно влияет на предсказание модели $f$. В некоторых случаях это может оказаться совсем не так, и модель $g$ не сможет хорошо обучиться (функция потерь остенется высокой). Например, в случае изображений это может означать, что мы не можем линейно влиять на предсказание модели, удаляя отдельные суперпиксели. Возможно, модель ориентируется не на отдельные объекты, а на цвет изображения в целом. Тогда можно попробовать использовать другое упрощенное представление, элементами которого является информация об усредненном цвете изображения.

Работа алгоритма LIME не зависит от вида модели $f$ (нейронная сеть, решающие деревья и т. д.) и никак явно не использует информацию о том, как модель устроена "изнутри", то есть LIME является "model-agnostic" алгоритмом интерпретации.

### Примеры и обсуждение

На *рис. 2* мы видим объяснение предсказания сверточной нейронной сети Inception ([Szegedy et al., 2014]($Going Deeper with Convolutions$)). Сначала мы рассматриваем выходной нейрон, соответствующий классу "Electric guitar", и пытаемся аппроксимировать значение на этом нейроне с помощью линейной модели $g$, которая использует информацию о наличии или отстутствии суперпикселей ($M$ бинарных признаков, где $M$ - количество суперпикселей). В результате для каждого суперпикселя мы получаем вес, то есть вклад этого суперпикселя в предсказание "Electric guitar", и выделяем суперпиксели с наибольшим весом. Далее повторяем тот же алгоритм для двух других выходных нейронов, соответствующих классам "Acoustic guitar" и "Labrador". Как можно видеть из примера, алгоритм LIME концептуально достаточно прост.

<img src="assets/lime.jpg" width="800" align="center">

<center><i>Рис. 2. Пример результатов, полученных с помощью метода LIME. Отмечены суперпиксели с наибольшими весами в линейной объясняющей модели.</i></center>

Работа алгоритма LIME определяется выбором набора изменений $\Delta_i$, этот выбор осуществляется вручную и в каких-то случаях может быть неудачным. Поэтому LIME можно рассматривать как метод, в котором мы сначала формулируем гипотезу о том, как можно было бы объяснить предсказание модели, а затем проверяем ее.

Интересно было бы попробовать применить алгоритм LIME для интерпретации предсказаний человека на различных задачах. Такой подход мог бы дать лучшее понимание плюсов, минусов и границ применимости алгоритма.

Один из возможных минусов заключается в том, что измененные примеры $x_0 + \Delta x$ могут быть неестественными, ненатуральными (например, изображение, в котором стерта часть суперпикселей). Модель $f$, напротив, обучалась только на натуральных примерах, и чаще всего от нее не требуется корректная работа на ненатуральных примерах (не лежащих в многообразии исходных данных, в терминологии из [Fefferman et al., 2013]($Testing the Manifold Hypothesis$)).

Конечно хотелось бы проверять работу $f$ только для тех $x_0 + \Delta x$, которые также выглядят натурально. В табличных данных для более надежной интерпретации можно искать натуральные примеры, похожие на $x_0$ (если датасет имеет достаточный размер) и изучать работу модели на этих примерах (то есть сопоставлять изменение в предсказании модели и изменение входных данных относительно $x_0$.

В компьютерном знении нас может интересовать, каким образом модель предсказывает на изображении собаку. Вместо удаления случайных суперпикселей (что сделает изображение ненатуральным) можно изучить работу модели на видео, в котором переход между кадрами соответствует переходам между разными натуральными изображениями. Можно даже записать видео, по очереди закрывая отдельные части интересующего нас объекта. Также можно попробовать использовать компьютерную графику для генерации натуральных изображений, хотя такой подход сложен и может повлечь много новых проблем.

Конечно, можно разрабатывать более эффективные способы интерпретации для конкретных предметных областей, но особенность LIME именно в том, что это очень общий подход, который может быть применен к широкому классу моделей. При этом многие детали в нем, в частности вид упрощенного представления, могут быть выбраны произвольно.

### LIME-SP для глобальной интерпретации моделей

Авторы также предлагают надстройку над алгоритмом LIME, называемую submodular pick (SP), которая может помочь интерпретировать модель в целом, а не только на конкретном примере. Для этого выбирается набор тестовых примеров, и каждый пример интерпретируется алгоритмом LIME, при этом к каждому примеру мы применяем одни и те же по смыслу изменения. Используя линейную модель $g$, в результате мы получаем матрицу $W$, строкой которой является номер примера, столбцом - позиция в векторе упрощенного представления, значением - вес данной позиции на данном примере.

Например, в случае модели, работающей с текстом и использующей bag-of-words, столбец матрицы $W$ будет соответствовать номер слова в словаре. Однако с суперпикселями так не получится, поскольку на каждом тестовом примере суперпиксели разные по смыслу.

Получив матрицу $W$, мы можем рассчитать глобальный вес каждой позиции, найдя норму каждого столбца. Также мы можем попытаться выбрать небольшой набор строк матрицы $W$ такой, чтобы в этом наборе для каждого столбца хотя бы раз встретилось большое значение. Этот набор строк соответствует набору тестовых примеров, которых предположительно может быть достаточно для глобальной интерпретации модели.

## SHAP: Shapley Additive Explanation Values

В данном разделе мы рассмотрим подход SHAP ([Lundberg and Lee, 2017]($A Unified Approach to Interpreting Model Predictions$)), позволяющий оценивать важность признаков в произвольных моделях машинного обучения, а также может быть применен как частный случай метода LIME.

### Shapley values

SHAP основан на [теории игр](https://ru.wikipedia.org/wiki/%D0%A2%D0%B5%D0%BE%D1%80%D0%B8%D1%8F_%D0%B8%D0%B3%D1%80) - области математики, изучающей взаимодействие (игру) между игроками, преследующими некие цели и действующими по неким правилам. [Кооперативной игрой](https://ru.wikipedia.org/wiki/%D0%9A%D0%BE%D0%BE%D0%BF%D0%B5%D1%80%D0%B0%D1%82%D0%B8%D0%B2%D0%BD%D0%B0%D1%8F_%D1%82%D0%B5%D0%BE%D1%80%D0%B8%D1%8F_%D0%B8%D0%B3%D1%80) называется такая игра, в которых группа игроков действует совместно. С середины XX века ([Shapley, 1952]($A Value for N-Person Games$)) известны так называмые [Shapley values](https://ru.wikipedia.org/wiki/%D0%92%D0%B5%D0%BA%D1%82%D0%BE%D1%80_%D0%A8%D0%B5%D0%BF%D0%BB%D0%B8), которые позволяют численно оценить вклад каждого игрока в достижение общей цели. Таким образом, shapley value - это отображение игрок $\to$ число, рассчитываемое по определенной формуле. Эта формула задается аксиоматически, то есть формулируется ряд необходимых свойств и доказывается, что существует единственное решение, которое им удовлетворяет.

Этот же подход можно применить в машинном обучении, если игроками считать отдельные признаки, а результатом игры - ответ модели на конкретном примере $x$. **Shapley regression values** ([Lipovetsky and Conklin, 2001]($Analysis of Regression in Game Theory Approach$)) позволяют оценить вклад каждого признака в ответ модели $f(x)$ на конкретном примере $x$. Обозначим за $F$ все имеющееся множество признаков, $S$ - некое подмножество признаков, $x_S$ - признаки из множества $S$ для примера $x$, $f_S$ - модель, обученная на признаках из множества $S$. Тогда shapley regression value $\phi_i$ для признака $i$ на примере $x$ рассчитывается по следующей формуле:

$\normalsize \phi_i = \sum\limits_{S \subseteq F \setminus \{i\}} \cfrac{|S|!(|F| - |S| - 1)!}{|F|!}\ (f_{S \cup \{i\}}(x_{S \cup \{i\}}) - f_S(x_S)) \tag{1}$

Суммирование выполняется по всем возможным подмножествам признаков, не содержащим признака $i$. Для каждого подмножества $S$ мы считаем разность между ответом модели на признаках $S \cup \{i\}$ и ответом модели на признаках $S$. Иными словами, мы считаем, насколько повлияет на ответ модели добавление признака $i$. Также при суммировании мы используем веса, которые принимают наибольшие значения при $|S| \approx 0$ или $|S| \approx |F|$ и наименьшие значения при $|S| \approx |F|\ /\ 2$.

Минусом такого подхода является высокая сложность вычислений: мы считаем сумму $2^{|F|-1}$ слагаемых, и при этом в каждом слагаемом нужно заново обучать модель на выбранном подмножестве признаков. Однако мы можем существенно быстрее посчитать приближенные значения $\phi_i$, называемые **shapley sampling values**:

1. Мы можем считать не все элементы суммы $(1)$, а лишь некоторые, обладающие большими весами. Для этого мы можем использовать веса как вероятности при семплировании элементов суммы (аппроксимация семплированием), что позволяет рассчитывать меньше, чем $2^{|F|-1}$ слагаемых.
2. Мы можем избавиться от необходимости обучать модель заново на каждом подмножестве признаков. Пусть мы имеем датасет, взятый из порождающего распределения $\mathcal{D}$, и на всех признаках обучили модель $f$. Модель, использующая только признаки из подмножества $S$, моделирует мат. ожидание по $\mathcal{D}$ при условии заданных значений $x_S$: $E_{x \sim \mathcal{D}}\ [y\ |\ x_S]$, и эту величину можно аппроксимировать, используя обучающий датасет.

### Additive feature attribution methods

Shapley regression values и Shapley sampling values были известны и раньше. Авторы SHAP ([Lundberg and Lee, 2017]($A Unified Approach to Interpreting Model Predictions$)) сопоставили данный подход с несколькими появившимися в 2010-х годах методами интерпретации моделей машинного обучения, которые используют локальную аппроксимацию, в первую очередь с LIME, а также с *layer-wise relevance propagation* ([Bach et al., 2015]($On Pixel-Wise Explanations for Non-Linear Classifier Decisions by Layer-Wise Relevance Propagation$)) и *activation difference propagation (DeepLIFT)* ([Shrikumar et al., 2017]($Learning Important Features Through Propagating Activation Differences$)), которые мы сейчас не будем рассматривать.

Как LIME, так и DeepLIFT рассматривают локальное упрощенное представление в виде вектора $z$ (где $z_i = 0$ означает наличие изменения $\Delta_i$ относительно примера $x_0$). В разделе, посвященное LIME, мы говорили о таком представлении и об объясняющей модели $g$, аппроксимирующей $f(h(z))$. Авторы SHAP рассматривают случай, когда объясняющая модель линейна (такие способы интерпретации авторы называют **additive feature attribution methods**):

$g(z) = \phi_0 + \sum\limits_{i=1}^M \phi_i z_i, \quad \phi_i \in \mathbb{R}, z_i \in \{0, 1\} \tag{2}$

Важным является случай, когда $\Delta_i$ означает удаление некоего признака из $x_0$, то есть при $z_i = 0$ один из признаков в $x_0$ заменяется на неопределенное значение. В этом случае будем говорить, что $z_i$ *задает наличие признака*. Тогда интерпретация модели $g$ позволить понять, насколько данный признак важен на данном примере $x_0$. Часто именно для этого SHAP применяется на практике (см. далее).

Однако если $z_i$ задает наличие признака, то прямой подсчет $f(h(z))$ может быть невозможен, так как $h(z)$ будет содержать пропущенные значения, с которыми многие модели не умеют работать. Обозначим за $x_S$ множество известных признаков, то есть таких, для которых $z_i = 1$ (тогда значения этих признаков берутся из $x_0$). Значение $f(h(z))$ мы можем аппроксимировать [условным мат. ожиданием](https://ru.wikipedia.org/wiki/%D0%A3%D1%81%D0%BB%D0%BE%D0%B2%D0%BD%D0%BE%D0%B5_%D0%BC%D0%B0%D1%82%D0%B5%D0%BC%D0%B0%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5_%D0%BE%D0%B6%D0%B8%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5) $E [f(x)\ |\ x_S]$ (по порождающему распределению данных, из которого взята обучающая выборка) при условии значений тех признаков, которые известны. Модель $g$ линейна, то есть основана на предположении, что все признаки влияют на ответ линейно и независимо, поэтому неизвестные признаки мы можем заменить мат. ожиданием по порождающему распределению, которое приближаем с помощью среднего значения по обучающей выборке (см. также раздел 4 в [Lundberg and Lee, 2017]($A Unified Approach to Interpreting Model Predictions$)). Таким образом, если $z_i$ задает наличие признака, то в случае $z_i = 0$ мы заменяем признак средним значением по обучающей выборке.

### Shapley additive explanation values

Теперь мы можем описать способ рассчета важности бинарных признаков $z_i$ в модели $f$ на тестовом примере $x_0$, используя Shapley values, которые обозначаются как $\phi_i(f, x_0)$ и используются в модели $g$ (формула 2). Тогда мы сможем оценить важность каждого изменения $\Delta_i$, и таким образом интерпретировать предсказание модели на данном примере. Чтобы вывести формулу для $\phi_i(f, x_0)$, авторы вводят несколько требуемых свойств, сходных по смыслу с используемыми в теории игр ([Shapley, 1952]($A Value for N-Person Games$)).

Следуя [Lundberg and Lee, 2017]($A Unified Approach to Interpreting Model Predictions$), далее мы будем считать, что вектор упрощенного представления $z_0$ для исходного примера $x_0$ не обязательно состоит полностью из единиц. Например, если $z_i$ означает наличие $i$-го признака в $x_0$, то в нем может и отсутствовать этот признак.

**Свойство 1** (local accuracy). На примере $x_0$ ответы моделей $f$ и $g$ должны совпадать: $g(z_0) = f(x_0)$.

**Свойство 2** (missingness). Если $z_i$ задает наличие некоего признака (см. выше), и этот признак отсутствует в $x_0$, то $\phi_i(f, x_0) = 0$, то есть важность данного признака равна нулю. Здесь нужно подчеркнуть, что речь идет об интерпретации предсказания модели $f$ на примере $x_0$, тогда как на других примерах этот признак может иметь ненулевую важность.

**Свойство 3** (consistency). Ранее мы рассматривали только одну модель $f$, теперь рассмотрим две произвольные модели $f$ и $f^\prime$, работающие с одними и теми же входными данными. Если добавление $i$-го компонента в $z$ ($z_i = 0 \Rightarrow z_i = 1$) приводит к не меньшему росту выходного значения для модели $f$, чем для модели $f^\prime$, при всех возможных значениях других компонентов в $z$, то важность данного компонента в модели $f$ не меньше, чем чем в модели $f^\prime$: $\phi_i(f, x_0) \geq \phi_i(f^\prime, x_0)$. Более формально описание третьего свойства см. в [Lundberg and Lee, 2017]($A Unified Approach to Interpreting Model Predictions$), раздел 3. Фактически добавление $i$-го компонента означает удаление $i$-го изменения и эквивалентно добавлению $i$-го игрока в теории игр. Например, если $z_i$ означают наличие суперпикселей, то каждый суперпиксель - игрок. Если же $z_i$ задают наличие признаков, тогда каждый признак - игрок. При этом задача Shapley values - оценить вклад каждого из игроков в результат.



Введем теперь следующие обозначения:

- $z \subseteq z_0$ - все такие бинарные векторы $z$, множество единиц которых является подмножеством множества единиц в $z_0$ (то есть если на какой-то позиции в $z_0$ стоит ноль, то во всех $z \subseteq z_0$ там тоже стоит ноль). Если, например, вектор $z_0$ состоит целиком из единиц, то $z \subseteq z_0$ - все возможные бинарные векторы той же размерности.
- $|z|$ - количество единиц в векторе $z$
- $z \setminus i$ - вектор $z$, в котором $i$-й элемент заменен нулем
- За $M$ по прежнему будем обозначать размерность вектора $z$

Авторы SHAP доказывают, что единственный способ задания $\phi_i(f, x_0)$, удовлетворяющий всем описанным свойствам, выглядит следующим образом:

$\normalsize \phi_i(f, x_0) = \sum\limits_{z \subseteq z_0} \cfrac{|z|! (M - |z| - 1)}{M!}\ (f(h(z)) - f(h(z \setminus i))) \tag{3}$

Если $z_i$ задает наличие признака, то в $f(h(z))$ отсутствующие признаки заменяются средними значениями по обучающей выборке (об этом см. выше в разделе "Additive feature attribution methods").

Данная формула имеет большое сходство с Shapley regression values $(1)$. Здесь хорошо видна связь с теорией игр: в теории игр Shapley value для $i$-го игрока оценивается путем добавления $i$-го игрока во все возможные подмножества игроков и подсчета вклада данного игрока, то есть разности результата с ним и без него. В даном же случае мы добавляем $i$-й компонент в вектор упрощенного представления при всех возможных значениях других компонентов и подсчитываем изменение в ответе модели $f$ от такого добавления.

### Kernel SHAP: Linear LIME + Shapley kernel

Вспомним, что в LIME при выборе линейной модели $g$ мы затем искали ее веса $\phi_i$, но для этого мы должны были выбрать функцию потерь $\mathcal{L}$, метрику сходства $\pi$ и штраф сложности $\Omega$. Авторы SHAP выводят единственно возможные значения для $\mathcal{L}$, $\pi$ и $\Omega$ такие, что полученные веса $\phi_i$ будут равны Shapley additive explanation values $\phi_i(f, x_0)$, рассчитанным по формуле $(3)$, при обучении на всех возможных $z$. Если же мы будем обучать $g$ не на всех $z$, то полученные веса будут аппроксимировать $\phi_i(f, x_0)$.

- $\mathcal{L}$ - среднеквадратичное отклонение
- $\pi(z) = \cfrac{M-1}{{{M}\choose{|z|}}\ |z|\ (M-|z|)}$, где ${M}\choose{|z|}$ - [биномиальный коээфициент](https://en.wikipedia.org/wiki/Binomial_coefficient)
- $\Omega(g) = 0$

Основное отличие от примера, предложенного авторами LIME ([Ribeiro et al., 2016]($Why Should I Trust You?: Explaining the Predictions of Any Classifier$)), состоит именно в $\pi(z)$. Как мы видели в разделе "Объясняющая модель", $\pi(z)$ фактически является метрикой сходства между векторами, такие метрики часто называют ядрами (kernels), поэтому предложенная формула для $\pi(z)$ получила название Shapley kernel.

Shapley kernel назначает большие веса примерам как с большим количеством единиц, так и с большим количеством нулей.  Это означает, что мы обращаем основное внимание на примеры с (почти) максимальным и (почти) минимальным количеством компонентов (игроков). На *рис. 3* показано отличие Shapley kernel от L2 distance и cosine similarity, которые предлагалось использовать в LIME.

<img src="assets/shap.jpg" width="400" align="center">

<center><i>Рис. 1. Shapley kernel в сравнении с L2 distance и cosine similarity.</i></center>

----

Рассмотрим снова упрощенное представление $z$, описывающее изменения $\Delta_i$ относительно примера $x_0$.

- Наличием $i$-го игрока считается ситуация, когда $z_i = 1$ (то есть изменение $\Delta_i$ отсутствует)
- Отсутствием $i$-го игрока считается ситуация, когда $z_i$ неизвестно.

Используя LIME, мы создавали для входных данных модели $f$ упрощенное представление, и таким образом упрощали формат входных данных модели. То есть интерпретация модели $f$ осуществлялась в 2 шага:

1. Выбрать набор изменений $\Delta_i$ и создать локальное упрощенное представление $z$ для входных данных
2. Интерпретировать более простую модель

- Схема из кругов: интерпретация моделей со сложными признаками, с простыми признаками (переход - выбор конкретного примера, т. е. локальная интерпретация), LIME, SHAP, библиотека SHAP, saliency maps, mean impurity decrease.

- Интерпретация сложной модели = локальное упрощение модели + оценка важности признаков в простой модели. LIME - подход к упрощению модели, SHAP - подход к анализу вклада признаков в модели, в котором для вычислительной эффективности требуется простота модели.

- Локальное упрощение модели: терминология, обозначения и примеры